In [1]:
# from translate import Translator
from langdetect import detect, detect_langs
from dateutil import parser
import re
import psycopg2
import pandas as pd

import translators as ts
from os import environ

/var/folders/tg/c_k1syfs2b9gvpkyjkcmdqx00000gn/T/ipykernel_71768/21036820.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
use_date_str = False

In [3]:
conn = psycopg2.connect(
host = environ['DB_HOST'],
port = environ['DB_PORT'],
user = environ['DB_USER'],
password = environ['DB_PASSWORD'],
database = environ['DB_NAME'])

In [4]:
limit = 500

query = r"""
    SELECT a.id, a.iso3, a.url
    FROM articles a
;"""

body = pd.read_sql_query(query, conn)
body

/var/folders/tg/c_k1syfs2b9gvpkyjkcmdqx00000gn/T/ipykernel_71768/2544918755.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  body = pd.read_sql_query(query, conn)


,id,iso3,url
0,16219,None,https://www.undp.org/acceleratorlabs/blog/luz-...
1,4828,None,https://www.undp.org/policy-centre/singapore/c...
2,4826,None,https://www.undp.org/policy-centre/singapore/b...
3,4825,None,https://www.undp.org/policy-centre/singapore/c...
4,4815,None,https://www.undp.org/policy-centre/singapore/b...
...,...,...,...
13169,12287,TUR,https://www.undp.org/turkiye/publications/2001...
13170,13215,None,https://www.undp.org/papp/publications/climate...
13171,13167,None,https://www.undp.org/myanmar/publications/stat...
13172,12999,None,https://www.undp.org/es/costa-rica/publication...


In [5]:
country_list = {
"Algeria": "DZA",
"Angola": "AGO",
"Argentina": "ARG",
"Azerbaijan": "AZE",
"Bangladesh": "BGD",
"Barbados": "BRB",
"Belarus": "BLR",
"Benin": "BEN",
"Bhutan": "BTN",
"Bosnia and Herzegovina": "BIH",
"bosnia-herzegovina": "BIH",
"Bolivia": "BOL",
"Burkina Faso": "BFA",
"burkina-faso": "BFA",
"Cambodia": "KHM",
"Cameroon": "CMR",
"Cape Verde": "CPV",
"Chad": "TCD",
"Colombia": "COL",
"Congo": "COG",
"Ivory Coast": "CIV",
"cote-d-ivoire": "CIV",
"cote_divoire": "CIV",
"Dominican Republic": "DOM",
"dominican-republic": "DOM",
"Democratic Republic of Congo": "COD",
"drcongo": "COD",
"Ecuador": "ECU",
"Egypt": "EGY",
"El Salvador": "SLV",
"el-salvador": "SLV",
"Eswatini": "SWZ",
"Ethiopia": "ETH",
"Fiji": "FJI",
"Gambia": "GMB",
"Georgia": "GEO",
"Ghana": "GHA",
"Guatemala": "GTM",
"Guinea": "GIN",
"Guinea-Bissau": "GNB",
"guinea_bissau": "GNB",
"Haiti": "HTI",
"India": "IND",
"Indonesia": "IDN",
"Iraq": "IRQ",
"Jordan": "JOR",
"Kazakhstan": "KAZ",
"Kenya": "KEN",
"Kyrgyzstan": "KGZ",
"Laos": "LAO",
"laopdr": "LAO",
"Lebanon": "LBN",
"Lesotho": "LSO",
"Libya": "LBY",
"Malawi": "MWI",
"Malaysia": "MYS",
"Maldives": "MDV",
"Mali": "MLI",
"Mauritania": "MRT",
"Mauritius": "MUS",
"mauritius-seychelles": "MUS",
"Mexico": "MEX",
"Mongolia": "MNG",
"Morocco": "MAR",
"Mozambique": "MOZ",
"Namibia": "NAM",
"Nepal": "NPL",
"Niger": "NER",
"Nigeria": "NGA",
"North Macedonia": "MKD",
"north-macedonia": "MKD",
"Pakistan": "PAK",
"Palestine": "PSE",
"Panama": "PAN",
"Paraguay": "PRY",
"Peru": "PER",
"Philippines": "PHL",
"Rwanda": "RWA",
"Samoa": "WSM",
"Saudi Arabia": "SAU",
"Saudi-Arabia": "SAU",
"Senegal": "SEN",
"Serbia": "SRB",
"Sierra-Leone": "SLE",
"Somalia": "SOM",
"South Africa": "ZAF",
"South-Africa": "ZAF",
"South Sudan": "SSD",
"South-Sudan": "SSD",
"Sudan": "SDN",
"Syria": "SYR",
"Tanzania": "TZA",
"Thailand": "THA",
"Timor Leste": "TLS",
"Timor-Leste": "TLS",
"Togo": "TGO",
"Trinidad and Tobago": "TTO",
"trinidad-and-tobago": "TTO",
"Tunisia": "TUN",
"Turkey": "TUR",
"Turkiye": "TUR",
"Uganda": "UGA",
"Ukraine": "UKR",
"Uruguay": "URY",
"Uzbekistan": "UZB",
"Vietnam": "VNM",
"Zambia": "ZMB",
"Zimbabwe": "ZWE",
"acceleratorlabs": "NUL",
}

In [6]:
exclude = [
    "singapore",
    "latin-america",
    "sgtechcenter",
    "africa",
    "cyprus",
    "kuwait",
    "moldova",
    "tajikistan",
    "yemen",
    "suriname",
    "costa-rica",
    "botswana",
    "armenia",
    "cuba",
    "srilanka",
    "honduras",
    "myanmar",
    "istanbul",
    "turkmenistan",
    "papp",
    "chile",
    "seoul",
    "china",
    "djibouti",
    "central-african-republic",
    "brazil",
    "romecentre",
    "sao-tome-principe",
    "burundi",
    "asia-pacific",
    "papua-new-guinea",
    "equatorial-guinea",
    "eurasia",
    "denmark",
    "japan",
    "liberia",
    "norway",
    "oslo",
    "montenegro",
    "venezuela",
    "germany",
    "sweden",
    "gabon",
    "afghanistan",
    "madagascar",
    "zskgke326",
    "arab-states",
    "nairobi",
    "comoros",
    "finland",
    "iran",
    "kosovo",
    "european-union",
    "nicaragua",
    "albania",
    "pacific",
    "geneva",
    "jamaica",
    "sgtechcentre",
    "eritrea",
    "guyana",
    "bahrain",
    "belize",
    "seoul-policy-centre",
    "united-states"
]

In [8]:
def splitURL (row):
    components = row['url'].replace('//','/').split('/')[2:-1]
    return components
def extractCountry (row):
    intersect = list(set(row['url_components']) & set([x.lower() for x in list(country_list.keys())]))
    return intersect
def excludeCountry (row):
    intersect = list(set(row['url_components']) & set([x.lower() for x in exclude]))
    return intersect
def mapISO3 (row):
    if len(row['country']) > 0: iso3 = { k.lower(): v for k, v in country_list.items() }[row['country'][0]]
    else: iso3 = None
    return iso3

body['url_components'] = body.apply(splitURL, axis = 1)
body['country'] = body.apply(extractCountry, axis = 1)
body['url_iso3'] = body.apply(mapISO3, axis = 1)
body['exclude'] = body.apply(excludeCountry, axis = 1)
body[(body['country'].map(len) == 0) & (body['exclude'].map(len) == 0) & (body['url_components'].map(len) > 1)].reset_index()

,index,id,iso3,url,url_components,country,url_iso3,exclude
0,47,12482,None,https://www.undp.org/es/publications/resultado...,"[es, publications]",[],None,[]
1,438,12495,None,https://www.undp.org/fr/publications/changemen...,"[fr, publications]",[],None,[]
2,442,12436,None,https://www.undp.org/es/publications/inforapid...,"[es, publications]",[],None,[]
3,745,12470,None,https://www.undp.org/fr/publications/evaluatio...,"[fr, publications]",[],None,[]
4,756,12485,None,https://www.undp.org/es/publications/resultado...,"[es, publications]",[],None,[]
...,...,...,...,...,...,...,...,...
182,13027,12500,None,https://www.undp.org/fr/publications/gouvernan...,"[fr, publications]",[],None,[]
183,13028,12520,None,https://www.undp.org/fr/publications/evaluatio...,"[fr, publications]",[],None,[]
184,13030,12524,None,https://www.undp.org/fr/publications/haiti-un-...,"[fr, publications]",[],None,[]
185,13037,12743,None,https://www.undp.org/fr/publications/inforapid...,"[fr, publications]",[],None,[]


In [9]:
body[body['country'].map(len) != 0].reset_index()

,index,id,iso3,url,url_components,country,url_iso3,exclude
0,0,16219,None,https://www.undp.org/acceleratorlabs/blog/luz-...,"[acceleratorlabs, blog]",[acceleratorlabs],NUL,[]
1,6,16221,USA,https://www.undp.org/egypt/blog/feeding-future...,"[egypt, blog]",[egypt],EGY,[]
2,11,16223,None,https://www.undp.org/acceleratorlabs/blog/insp...,"[acceleratorlabs, blog]",[acceleratorlabs],NUL,[]
3,12,15791,ARG,https://www.undp.org/es/argentina/publicacione...,"[es, argentina, publicaciones]",[argentina],ARG,[]
4,13,11204,SYR,https://www.undp.org/syria/our-leadership,[syria],[syria],SYR,[]
...,...,...,...,...,...,...,...,...
7386,13165,12288,TUR,https://www.undp.org/tr/turkiye/publications/2...,"[tr, turkiye, publications]",[turkiye],TUR,[]
7387,13166,12279,TUR,https://www.undp.org/tr/turkiye/publications/1...,"[tr, turkiye, publications]",[turkiye],TUR,[]
7388,13167,12321,TUR,https://www.undp.org/tr/turkiye/publications/2...,"[tr, turkiye, publications]",[turkiye],TUR,[]
7389,13168,12299,TUR,https://www.undp.org/tr/turkiye/publications/u...,"[tr, turkiye, publications]",[turkiye],TUR,[]


In [10]:
# translator = Translator(from_lang = 'es', to_lang = 'en')
# translator.translate(r'21 DE FEBREERO DE 2022')
# ts.translate_text('2 DE MAYO DE 2023', from_language = 'es', to_language = 'en')

In [11]:
cur = conn.cursor()

for i, r in body[body['url_iso3'].notnull()].iterrows():
    sql = """
        UPDATE articles
        SET iso3 = %s
        WHERE id = %s
    ;"""
    cur.execute(sql, (r['url_iso3'], r['id']))

conn.commit()
cur.close()

In [21]:
## CLOSE THE CONNECTION TO THE DB
conn.close()